In [3]:
with open("test_input_1.txt", "r") as f:
    test_input_1 = f.read()
with open("test_input_2.txt", "r") as f:
    test_input_2 = f.read()
with open("input.txt", "r") as f:
    real_input = f.read()
test_input_1


'LR\n\n11A = (11B, XXX)\n11B = (XXX, 11Z)\n11Z = (11B, XXX)\n22A = (22B, XXX)\n22B = (22C, 22C)\n22C = (22Z, 22Z)\n22Z = (22B, 22B)\nXXX = (XXX, XXX)'

In [4]:
def get_steps(raw_input):
    return raw_input.split("\n")[0]

get_steps(test_input_1)

'LR'

In [5]:
def get_route(raw_input):
    return raw_input.split("\n\n")[1].split("\n")

get_route(test_input_1)


['11A = (11B, XXX)',
 '11B = (XXX, 11Z)',
 '11Z = (11B, XXX)',
 '22A = (22B, XXX)',
 '22B = (22C, 22C)',
 '22C = (22Z, 22Z)',
 '22Z = (22B, 22B)',
 'XXX = (XXX, XXX)']

In [6]:
from collections import namedtuple
Node = namedtuple("Node", "l r")

In [7]:
def build_node(line):
    node, _, raw_left, raw_right = line.split()
    left = raw_left[1:-1]
    right = raw_right[:-1]
    return {node:Node(left, right)}

build_node(get_route(test_input_1)[0])

{'11A': Node(l='11B', r='XXX')}

In [8]:
def build_graph(lines):
    graph = dict()
    for line in lines:
        graph = graph | build_node(line)
    return graph

build_graph(get_route(test_input_1))

{'11A': Node(l='11B', r='XXX'),
 '11B': Node(l='XXX', r='11Z'),
 '11Z': Node(l='11B', r='XXX'),
 '22A': Node(l='22B', r='XXX'),
 '22B': Node(l='22C', r='22C'),
 '22C': Node(l='22Z', r='22Z'),
 '22Z': Node(l='22B', r='22B'),
 'XXX': Node(l='XXX', r='XXX')}

In [9]:
from itertools import cycle
START_NODE = "AAA"
END_NODE = "ZZZ"
def calculate_steps_to_end(graph, steps):
    current_node = graph[START_NODE]
    for i, step in enumerate(cycle(steps)):
        new_node_id = current_node.l if step == "L" else current_node.r

        if new_node_id == END_NODE:
            return i+1
        
        current_node = graph[new_node_id]

calculate_steps_to_end(build_graph(get_route(test_input_1)), get_steps(test_input_1))

KeyError: 'AAA'

In [10]:
def get_result_1(raw_input):
    return calculate_steps_to_end(build_graph(get_route(raw_input)), get_steps(raw_input))

get_result_1(test_input_2)

6

In [11]:
get_result_1(real_input)

19099

In [12]:

def get_start_nodes(graph):
    return [id for id in graph.keys() if id[-1]=="A"]

get_start_nodes(build_graph(get_route(real_input)))

['HVA', 'LBA', 'FXA', 'GHA', 'PSA', 'AAA']

In [37]:
def get_advance_function(graph, steps):
    loop_cache = {}

    def get_steps_to_Z(start_node_id, start_step=0):
        nonlocal graph, steps, loop_cache
        step_start_index = start_step % len(steps)
        try:
            new_node_id, loop_size = loop_cache[(start_node_id, step_start_index)]
            return new_node_id, start_step + loop_size
        except KeyError:
            print("nocache ", end="")

        current_node = graph[start_node_id]
        steps = steps[start_step:] + steps[:start_step]

        for i, step in enumerate(cycle(steps)):
            new_node_id = current_node.l if step == "L" else current_node.r

            if new_node_id[-1] == "Z":
                loop_cache[(start_node_id, step_start_index)] = new_node_id, i + 1
                return new_node_id, start_step + i + 1
            
            current_node = graph[new_node_id]
    
    return get_steps_to_Z

get_advance_function(build_graph(get_route(real_input)), get_steps(real_input))("AAA")

nocache 

('ZZZ', 19099)

In [38]:
def get_convergent_steps(graph, steps):
    start_nodes = get_start_nodes(graph)
    advance_function = get_advance_function(graph, steps)
    step_counts = [advance_function(id) for id in start_nodes]

    i=0
    while any(step_count != step_counts[0][1] for _, step_count in step_counts):
        if not i%1000000 or i<10:
            print(i, step_counts)
        i+=1
        to_advance = min(step_counts, key=lambda x: x[1])
        step_counts[step_counts.index(to_advance)] = advance_function(to_advance[0], start_step=to_advance[1])
    return step_counts[0][1]

def run(raw_input):
    return get_convergent_steps(build_graph(get_route(raw_input)), get_steps(raw_input))

run(test_input_1)
    
    


nocache nocache 0 [('11Z', 2), ('22Z', 3)]
nocache 1 [('11Z', 4), ('22Z', 3)]
nocache 2 [('11Z', 4), ('22Z', 6)]


6

In [40]:
from math import lcm
def get_simple_convergent_steps(graph, steps):
    start_nodes = get_start_nodes(graph)
    advance_function = get_advance_function(graph, steps)
    step_counts = [advance_function(id) for id in start_nodes]
    step_counts = [c for _, c in step_counts]
    
    return lcm(*step_counts)

def simple_run(raw_input):
    return get_simple_convergent_steps(build_graph(get_route(raw_input)), get_steps(raw_input))

simple_run(test_input_1)

nocache nocache 

6

In [41]:
simple_run(real_input)

nocache nocache nocache nocache nocache nocache 

17099847107071

In [39]:
run(real_input)

nocache nocache nocache nocache nocache nocache 0 [('NMZ', 15871), ('SJZ', 16409), ('GNZ', 21251), ('TNZ', 18023), ('BNZ', 12643), ('ZZZ', 19099)]
nocache 1 [('NMZ', 15871), ('SJZ', 16409), ('GNZ', 21251), ('TNZ', 18023), ('BNZ', 25286), ('ZZZ', 19099)]
nocache 2 [('NMZ', 31742), ('SJZ', 16409), ('GNZ', 21251), ('TNZ', 18023), ('BNZ', 25286), ('ZZZ', 19099)]
nocache 3 [('NMZ', 31742), ('SJZ', 32818), ('GNZ', 21251), ('TNZ', 18023), ('BNZ', 25286), ('ZZZ', 19099)]
nocache 4 [('NMZ', 31742), ('SJZ', 32818), ('GNZ', 21251), ('TNZ', 36046), ('BNZ', 25286), ('ZZZ', 19099)]
nocache 5 [('NMZ', 31742), ('SJZ', 32818), ('GNZ', 21251), ('TNZ', 36046), ('BNZ', 25286), ('ZZZ', 38198)]
nocache 6 [('NMZ', 31742), ('SJZ', 32818), ('GNZ', 42502), ('TNZ', 36046), ('BNZ', 25286), ('ZZZ', 38198)]
7 [('NMZ', 31742), ('SJZ', 32818), ('GNZ', 42502), ('TNZ', 36046), ('BNZ', 37929), ('ZZZ', 38198)]
8 [('NMZ', 47613), ('SJZ', 32818), ('GNZ', 42502), ('TNZ', 36046), ('BNZ', 37929), ('ZZZ', 38198)]
9 [('NMZ', 47

KeyboardInterrupt: 